In [1]:
import numpy as np
import math
from qiskit import QuantumCircuit, Aer, QuantumRegister, ClassicalRegister, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library.basis_change import QFT

In [35]:
def increasing_tuple(input_list):
    increasing_pairs = []
    for i in range(len(input_list)):
        for j in range(len(input_list)):
            if input_list[i]<= input_list[j]:
                increasing_pairs.append((input_list[i], input_list[j]))
            else:
                pass
    return increasing_pairs

def target_set_circuit(input_list):
    pairs = increasing_tuple(input_list)
    num_qubits = int(np.log2(max(input_list)))+1
    ts_circuit = QuantumCircuit(2*num_qubits+3)
    for j in range(len(pairs)):
        for i in range(num_qubits):
            if ''.join('1' if x == '0' else '0' for x in bin(pairs[j][0])[2:].zfill(num_qubits))[num_qubits-i-1] == '1':
                ts_circuit.x(i)
            if ''.join('1' if x == '0' else '0' for x in bin(pairs[j][1])[2:].zfill(num_qubits))[num_qubits-i-1] == '1':
                ts_circuit.x(num_qubits + i)
        ts_circuit.mcx(list(range(2*num_qubits)), 2*num_qubits+1)
        for i in range(num_qubits):
            if ''.join('1' if x == '0' else '0' for x in bin(pairs[j][0])[2:].zfill(num_qubits))[num_qubits-i-1] == '1':
                ts_circuit.x(i)
            if ''.join('1' if x == '0' else '0' for x in bin(pairs[j][1])[2:].zfill(num_qubits))[num_qubits-i-1] == '1':
                ts_circuit.x(num_qubits + i)
        ts_circuit.barrier(range(2*num_qubits+3))
    return ts_circuit

def quantum_add_circuit(qubits, target_number):
    qa_circuit = QuantumCircuit(2*qubits+1)
    qa_circuit.append(QFT(qubits+1, do_swaps=False), list(range(qubits, 2*qubits+1)))
    for j in range(qubits):
        for k in range(qubits+1-j):
            lamda = np.pi / (2 ** (k))
            qa_circuit.cp(lamda, j, k+j+qubits)
    qa_circuit.append(QFT(qubits+1,do_swaps=False).inverse(), list(range(qubits, 2*qubits+1)))
    for i in range(qubits+1):
            if ''.join('1' if x == '0' else '0' for x in bin(target_number)[2:].zfill(qubits+1))[qubits-i] == '1':
                qa_circuit.x(i+qubits)
    return qa_circuit

def oracle_circuit(target_number, input_list):
    num_qubits = int(np.log2(max(input_list)))+1
    ts_circuit = target_set_circuit(input_list)
    qa_circuit = quantum_add_circuit(num_qubits, target_number)
    o_circuit = QuantumCircuit(2*num_qubits+3)
    o_circuit.append(ts_circuit, range(2*num_qubits+3))
    o_circuit.append(qa_circuit, range(2*num_qubits+1))
    o_circuit.mcx(list(range(num_qubits, 2*num_qubits+1)), 2*num_qubits+2)
    o_circuit.cz(2*num_qubits+1,2*num_qubits+2)
    o_circuit.mcx(list(range(num_qubits, 2*num_qubits+1)), 2*num_qubits+2)
    o_circuit.append(qa_circuit.inverse(), range(2*num_qubits+1))
    o_circuit.append(ts_circuit.inverse(), range(2*num_qubits+3))
    return o_circuit

def diffusion_circuit(qubits):
    d_circuit = QuantumCircuit(2*qubits+1)
    d_circuit.h(range(2*qubits))
    d_circuit.x(range(2*qubits))
    d_circuit.h(2*qubits-1)
    d_circuit.mct(list(range(2*qubits-1)), 2*qubits-1) 
    d_circuit.h(2*qubits-1)
    d_circuit.x(range(2*qubits))
    d_circuit.h(range(2*qubits))
    return d_circuit


def prime_number_circuit(target_number, input_list):
    num_qubits = int(np.log2(max(input_list)))+1
    o_circuit = oracle_circuit(target_number, input_list)
    d_circuit = diffusion_circuit(num_qubits)
    pn_circuit = QuantumCircuit(2*num_qubits+3, 2*num_qubits)
    pn_circuit.h(range(2*num_qubits))
    pn_circuit.barrier(range(2*num_qubits+3))
    for j in range(num_qubits):
        pn_circuit.append(o_circuit, range(2*num_qubits+3))
        pn_circuit.append(d_circuit, range(2*num_qubits+1))
    pn_circuit.measure(list(range(2*num_qubits)),list(range(2*num_qubits)))
    return pn_circuit

def find_prime_numbers(target_number, input_list):
    simulator = Aer.get_backend('qasm_simulator')
    num_qubits = int(np.log2(max(input_list)))+1
    if target_number > 2*max(input_list):
        return print("The target number exceeds the maximum sum.")
    pn_circuit = prime_number_circuit(target_number, input_list)
    count_quantum_circuit = execute(pn_circuit, backend = simulator, shots = 10000).result().get_counts()
    max_binary_counts = 0
    max_binary_value = ''
    for key, item in count_quantum_circuit.items():
        if item > max_binary_counts:
            max_binary_counts = item
            max_binary_value = key
    a_value = int(max_binary_value[-num_qubits:],2)
    b_value = int(max_binary_value[:num_qubits],2)
    return (a_value, b_value)

In [36]:
find_prime_numbers(34, [2,3,5,7,11,13,17])

(17, 17)

In [18]:
#plot_histogram(save, sort='value_desc')

In [12]:
sorted_count_prime_number_circuit = sorted(save.items(), key=lambda x:x[1], reverse=True)
converted_dict = dict(sorted_count_prime_number_circuit)
print(converted_dict)

{'0110100011': 1102, '0101100101': 1073, '0010000011': 18, '1010101010': 16, '1111100100': 16, '0100101011': 16, '0000000011': 16, '1111110000': 16, '1111011101': 15, '1101111011': 15, '0111110100': 15, '1001111110': 15, '0011111111': 15, '1110011101': 15, '0011000100': 14, '0100001110': 14, '0010110111': 14, '1100101000': 14, '1100111001': 14, '1000001000': 14, '1001011010': 14, '0101101101': 14, '0001001001': 14, '1101100111': 14, '1100011000': 14, '0001100000': 14, '1101000011': 13, '0001000101': 13, '0100001011': 13, '1010011011': 13, '0111100100': 13, '1101101000': 13, '1001111111': 13, '1010111110': 13, '0010001111': 13, '1000011010': 13, '0000011100': 13, '0101110111': 13, '0010101000': 13, '0111110000': 13, '0001100010': 13, '1111101111': 13, '1001110101': 13, '1011010110': 13, '1000010101': 13, '1010110001': 13, '0111011101': 13, '1101100010': 12, '1111110110': 12, '1100001111': 12, '0100000010': 12, '1011111110': 12, '0010111000': 12, '1001000110': 12, '1011110001': 12, '0111